## Hybrid Retriever- Combining Dense And Sparse Retriever

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document



In [19]:
# Step 1: Sample documents
docs = [
    Document(page_content="ວິທະຍາສາດຄອມພິວເຕີແມ່ນສາຂາວິຊາທີ່ສຳຄັນໃນຍຸກປັດຈຸບັນ."),
    Document(page_content="ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີຈະຊ່ວຍໃຫ້ນັກສຶກສາມີທັກສະທີ່ຈຳເປັນ."),
    Document(page_content="ນັກສຶກສາສາມາດຫາວຽກເຮັດງານທຳໄດ້ງ່າຍຂຶ້ນຖ້າມີຄວາມຮູ້ດ້ານນີ້."),
    Document(page_content="ປັນຍາປະດິດ (Artificial Intelligence), ຫຼື AI, ແມ່ນລະບົບຄອມພິວເຕີທີ່ຖືກສ້າງຂຶ້ນເພື່ອຮຽນຮູ້, ຄິດ, ແລະແກ້ໄຂບັນຫາໄດ້ຄືກັບມະນຸດ."),
    Document(page_content="ປັນຍາປະດິດແມ່ນເຕັກໂນໂລຊີທີ່ກຳລັງພັດທະນາຢ່າງໄວວາ."),
    Document(page_content="AI ສາມາດນຳມາໃຊ້ໃນຫຼາຍຂະແໜງການ."),
    Document(page_content="ຫຼາຍບໍລິສັດໃຫຍ່ກຳລັງລົງທຶນໃນການພັດທະນາ AI."),
    Document(page_content="ສປປ ລາວແມ່ນປະເທດທີ່ມີວັດທະນະທຳອັນຍາວນານ."),
    Document(page_content="ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ."),
    Document(page_content="ຄົນລາວຄົນຈິງໃຈ.")
] 

# Step 2: Dense Retriever (FAISS + HuggingFace)
embedding_model = HuggingFaceEmbeddings(model_name="D:\\model\\BAAI-bge-m3")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever(search_kwargs={"k": 3})

In [20]:
### Sparse Retriever(BM25)
sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=3 ##top- k documents to retriever

## step 4 : Combine with Ensemble Retriever
hybrid_retriever=EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weight=[0.7,0.3]
)


In [21]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D3536CD590>, search_kwargs={'k': 3}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001D356B95220>, k=3)], weights=[0.5, 0.5])

In [22]:
# Step 5: Query and get results
query = "AI ຄືຍັງ ແລະ ສຳຄັນແນວໃດ"
results = hybrid_retriever.invoke(query) 

# Step 6: Print results
for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}")


🔹 Document 1:
AI ສາມາດນຳມາໃຊ້ໃນຫຼາຍຂະແໜງການ.

🔹 Document 2:
ປັນຍາປະດິດ (Artificial Intelligence), ຫຼື AI, ແມ່ນລະບົບຄອມພິວເຕີທີ່ຖືກສ້າງຂຶ້ນເພື່ອຮຽນຮູ້, ຄິດ, ແລະແກ້ໄຂບັນຫາໄດ້ຄືກັບມະນຸດ.

🔹 Document 3:
ຄົນລາວຄົນຈິງໃຈ.

🔹 Document 4:
ຫຼາຍບໍລິສັດໃຫຍ່ກຳລັງລົງທຶນໃນການພັດທະນາ AI.

🔹 Document 5:
ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ.


### RAG Pipeline with hybrid retriever

In [23]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [24]:
# Step 5: Prompt Template
prompt = PromptTemplate.from_template("""
Answer the question based on the context below.
Important: Response in Lao Language only.
Context:
{context}

Question: {input}
""")

## step 6-llm
llm=init_chat_model("groq:meta-llama/llama-4-maverick-17b-128e-instruct",temperature=0.4)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D3536CE5D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D3536CEFD0>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', temperature=0.4, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [25]:
### Create stuff Docuemnt Chain
document_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)

## create Full rAg chain
rag_chain=create_retrieval_chain(retriever=hybrid_retriever,combine_docs_chain=document_chain)
rag_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D3536CD590>, search_kwargs={'k': 3}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001D356B95220>, k=3)], weights=[0.5, 0.5]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context below.\nImportant: Response in Lao Language only.\nContext:\n{context}\n\nQuestio

In [26]:
# Step 9: Ask a question
query = {"input": "ຂ້ອຍຈຳເປັນຕ້ອງຮຽນ ບັນຍາປະດິດບໍ່"}
response = rag_chain.invoke(query)

# Step 10: Output
print("✅ Answer:\n", response["answer"])

print("\n📄 Source Documents:")
for i, doc in enumerate(response["context"]):
    print(f"\nDoc {i+1}: {doc.page_content}")

✅ Answer:
 ທ່ານຄວນຮຽນຮູ້ກ່ຽວກັບປັນຍາປະດິດເພາະມັນແມ່ນເຕັກໂນໂລຊີທີ່ກຳລັງພັດທະນາຢ່າງໄວວາ ແລະ ມັນຈະຊ່ວຍໃຫ້ທ່ານມີທັກສະທີ່ຈຳເປັນໃນການປະກອບອາຊີບໃນອະນາຄົດ. ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີ ແລະ ປັນຍາປະດິດຈະຊ່ວຍໃຫ້ທ່ານສາມາດແກ້ໄຂບັນຫາ ແລະ ປະກອບສ່ວນໃນການພັດທະນາປະເທດຂອງທ່ານ.

📄 Source Documents:

Doc 1: ປັນຍາປະດິດແມ່ນເຕັກໂນໂລຊີທີ່ກຳລັງພັດທະນາຢ່າງໄວວາ.

Doc 2: ຄົນລາວຄົນຈິງໃຈ.

Doc 3: ປັນຍາປະດິດ (Artificial Intelligence), ຫຼື AI, ແມ່ນລະບົບຄອມພິວເຕີທີ່ຖືກສ້າງຂຶ້ນເພື່ອຮຽນຮູ້, ຄິດ, ແລະແກ້ໄຂບັນຫາໄດ້ຄືກັບມະນຸດ.

Doc 4: ປະຊາຊົນລາວມີຄວາມເປັນກັນເອງແລະມີໃຈມັກງາມ.

Doc 5: ການເຂົ້າໃຈວິທະຍາສາດຄອມພິວເຕີຈະຊ່ວຍໃຫ້ນັກສຶກສາມີທັກສະທີ່ຈຳເປັນ.

Doc 6: ສປປ ລາວແມ່ນປະເທດທີ່ມີວັດທະນະທຳອັນຍາວນານ.
